## 06 Extras: Stacking & Blending Practice

Notebook contents were partially inspired by <a href="https://alexanderdyakonov.wordpress.com/2017/03/10/cтекинг-stacking-и-блендинг-blending/"> Alexander Diakonov Blog </a>*`

### Let's have a short recap

### Bagging:

* Why Decision Trees are good to be used for bagging?
* How to estimate feature importances using the contructed Random Forest?
* How out-of-bag error is calculated using the constructed Random Forest?

### Practice: use Blending to finish EDA practice on Titanic Kaggle dataset

In [ ]:
from IPython.display import Image

import warnings
warnings.simplefilter("ignore")

import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')

from matplotlib.colors import ListedColormap
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 10, 6
%matplotlib inline

import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
data=pd.read_csv('data/titanic_dataset/train.csv')
data.head()

#### Repeat the Data Pre-processing step

Save the Initials for people's names in the dataset

In [ ]:
# lets extract the Salutations
data['Initial']=0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.')

data['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],
    inplace=True)
data.groupby('Initial')['Age'].mean() #lets check the average age by Initials

Fill NaNs in `Age` column.

In [ ]:
## Assigning the NaN Values with the Ceil values of the mean ages
data.loc[(data.Age.isnull())&(data.Initial=='Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial=='Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial=='Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial=='Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial=='Other'),'Age']=46

In [ ]:
data.Age.isnull().any() #So no null values left finally 

Fill NaNs for `Embarked` column.

In [ ]:
data['Embarked'].fillna('S',inplace=True)

In [ ]:
# Finally No NaN values
data.Embarked.isnull().any()

#### Repeat the Feature Engineering step

Convert age values to age bands

In [ ]:
data['Age_band']=0
data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4
data.head(2)

Add `FamilySize` and `Alone` features.

In [ ]:
# family size
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp']
# aloneb
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1

Add `FareCat` for fare categories, based on fare distribution quantiles

In [ ]:
data['Fare_Range'] = pd.qcut(data['Fare'],4)
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

In [ ]:
data['Fare_cat']=0
data.loc[data['Fare']<=7.91,'Fare_cat']=0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat']=1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat']=2
data.loc[(data['Fare']>31)&(data['Fare']<=513),'Fare_cat']=3

Covnert string columns into numeric values

In [ ]:
data['Sex'].replace(['male','female'],[0,1],inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace=True)

Drop unnecessary columns

In [ ]:
data.drop(['Name','Age','Ticket','Fare','Cabin','Fare_Range','PassengerId'],axis=1,inplace=True)

In [ ]:
print(data.shape)

#### Predictive Modelling

In [ ]:
train,test=train_test_split(data, test_size=0.2, random_state=42, stratify=data['Survived'])

train_X=train[train.columns[1:]]
train_Y=train[train.columns[:1]]

test_X=test[test.columns[1:]]
test_Y=test[test.columns[:1]]

X=data[data.columns[1:]]
Y=data['Survived']

Let's have a logistic regression baseline

In [ ]:
baseline_model = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
baseline_model.fit(train_X, train_Y)
prediction_bl = baseline_model.predict(test_X)
print('The ROC-AUC of the Logistic Regression is',metrics.roc_auc_score(prediction_bl, test_Y))

In [ ]:
train_X.head()

In [ ]:
#importing all the required ML packages
from sklearn.linear_model import LogisticRegression    #logistic regression
from sklearn.ensemble import RandomForestClassifier    #Random Forest
from sklearn.neighbors import KNeighborsClassifier     #KNN
from sklearn.tree import DecisionTreeClassifier        #Decision Tree
from sklearn.model_selection import train_test_split   #training and testing data split
from sklearn import metrics                            #accuracy measure

In [ ]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
from sklearn.model_selection import GridSearchCV # grid search

kfold = KFold(n_splits=3, random_state=22)

xyz = []
roc_values_cv, std = [], []
roc_values_test = []
classifiers=['Logistic Regression', 'KNN', 'Decision Tree', 'Random Forest']
models=[
    LogisticRegression(C=1e2),
    KNeighborsClassifier(n_neighbors=9),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(n_estimators=30, max_depth=5),
]

for i in models:
    model = i
    cv_result = cross_val_score(model, train_X, train_Y, cv=kfold, scoring="roc_auc")
    cv_result = cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    roc_values_cv.append(cv_result)
    model.fit(train_X, train_Y)
    roc_auc_test = metrics.roc_auc_score(model.predict(test_X), test_Y)
    roc_values_test.append(roc_auc_test)
    
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std, 'Test': roc_values_test} ,index=classifiers)       
new_models_dataframe2

In [ ]:
plt.subplots(figsize=(12,6))
box=pd.DataFrame(roc_values_cv,index=[classifiers])
box.T.boxplot()

In [ ]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
plt.title('Average CV Mean Accuracy')
fig=plt.gcf()
fig.set_size_inches(8,5)
plt.show()

### Stacking and Blending Practice

Dataset is split into N parts (folds), then the base algorithms are iteratively fit to N-1 folds and generate predictions for the N-th fold. Those predictions are then treated as the new features for the given N-th fold. To generate such metafeatures for the testing set, base algorithms are fitted to the full training dataset instead.

<img src='pics/stacking-eng.png'>

Simple way of stacking - blending:
1. Split training set into two disjoint subsets
2. Train several base models using the first subset of the data
3. Generate predictions for the second subset using the trained base models, and vice versa
4. Using the predictions from 3. as features and the ground truth targets, train a classifier using such metafeatures (secod-level classifier)

<img src='pics/blending-eng.png'>

In [ ]:
# for repeatability
SEED = 10

# number of folds for out-of-fold predictions
NFOLDS = 3
kf = KFold(n_splits=NFOLDS, random_state=SEED)

train_X, train_Y = np.array(train_X), np.array(train_Y)
test_X, test_Y = np.array(test_X), np.array(test_Y)

In [ ]:
def get_oof(clf, x_train, y_train, x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))
    
    for i, (train_index, test_index) in enumerate(kf.split(x_train, y_train)):
        x_tr = x_train[train_index, :]
        y_tr = y_train[train_index]
        x_te = x_train[test_index, :]

        clf.fit(x_tr, y_tr)
        
        oof_train[test_index] = clf.predict_proba(x_te)[:, 1]
        oof_test_skf[i, :] = clf.predict_proba(x_test)[:, 1]

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [ ]:
model_lr = LogisticRegression(random_state=SEED)
model_knn = KNeighborsClassifier(n_neighbors=9)
model_dt = DecisionTreeClassifier(max_depth=5)
model_rf = RandomForestClassifier(n_estimators=100, random_state=SEED, max_depth=5)

In [ ]:
# Create our OOF train and test predictions. These base results will be used as new features
lr_oof_train, lr_oof_test = get_oof(model_lr, train_X, train_Y, test_X) # Logistic Regression
knn_oof_train, knn_oof_test = get_oof(model_knn, train_X, train_Y, test_X) # KNN
dt_oof_train, dt_oof_test = get_oof(model_dt, train_X, train_Y, test_X) # XGBoost
rf_oof_train, rf_oof_test = get_oof(model_rf, train_X, train_Y, test_X) # Random Forest
print("Training is complete")

In [ ]:
base_predictions_train = pd.DataFrame(
    {
        'LogReg': lr_oof_train.ravel(),
        'kNN': knn_oof_train.ravel(),
        'DecisionTree': dt_oof_train.ravel(),
        'RandomForest': rf_oof_train.ravel(),
        'GT': np.array(train_Y).ravel()
    }
)
base_predictions_train.head()

In [ ]:
x_train = np.concatenate((lr_oof_train, knn_oof_train, dt_oof_train, rf_oof_train), axis=1)
x_test = np.concatenate((lr_oof_test, knn_oof_test, dt_oof_test, rf_oof_test), axis=1)

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()

# Create the parameter grid
gbm_param_grid = {
    'n_estimators': range(5, 50, 5),
    'max_depth': [2, 5, 10, -1],
    'learning_rate': [.4, .5, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}

# Perform grid search
xgb_gs = GridSearchCV(
    estimator=xgb_model, 
    param_grid=gbm_param_grid,
    scoring="roc_auc",
    verbose=1,
    cv=3,
    n_jobs=-2
)

# Fit grid search to the data
xgb_gs.fit(x_train, train_Y)

# Print the best parameters and lowest RMSE
print("Best parameters found: ", xgb_gs.best_params_)
print("Best ROC-AUC found: ", xgb_gs.best_score_)

In [ ]:
xgb_best = XGBClassifier(**xgb_gs.best_params_)
xgb_best.fit(x_train, train_Y)

print("Test ROC-AUC: ", metrics.roc_auc_score(xgb_best.predict(x_test), test_Y))

### Notes: